In [66]:
#%%capture
!pip install yfinance==0.2.58
#!pip install yfinance
#!pip install twelvedata
import numpy as np
import pandas as pd
import yfinance as yf
!pip show yfinance

Name: yfinance
Version: 0.2.58
Summary: Download market data from Yahoo! Finance API
Home-page: https://github.com/ranaroussi/yfinance
Author: Ran Aroussi
Author-email: ran@aroussi.com
License: Apache
Location: /usr/local/lib/python3.11/dist-packages
Requires: beautifulsoup4, curl_cffi, frozendict, multitasking, numpy, pandas, peewee, platformdirs, pytz, requests
Required-by: 


In [67]:
ticker = "1120.SR"
"""
ticker = "1120:Tadawul"
from twelvedata import TDClient

# Initialize TDClient with your API key
td = TDClient(apikey="2ced3e308ffe4a2ea90b4e4ca728fa9d")

ts = td.time_series(
    symbol=ticker,
    interval="1day",
    outputsize=100  # Number of data points to retrieve
)

# Convert to pandas DataFrame for easier manipulation
df = ts.as_pandas()
"""
data = yf.download(ticker, start="2024-01-01", end="2025-06-23")
data

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,1120.SR,1120.SR,1120.SR,1120.SR,1120.SR
Date,,,,,
2024-01-01,83.158173,83.637194,82.870757,82.870757,2871681
2024-01-02,83.253975,83.253975,82.295931,83.158172,3495693
2024-01-03,81.146278,82.870760,81.146278,82.487541,4288840
2024-01-04,83.828796,83.828796,79.805017,80.475645,5486572
2024-01-07,85.074257,86.128103,84.020403,84.307819,5900413
...,...,...,...,...,...
2025-06-16,92.599998,93.400002,91.199997,91.199997,3653317
2025-06-17,91.699997,93.099998,91.500000,92.599998,3959192


In [68]:
# Find the lowest price for the stock over the last week
data["prv_low_1"] = data["Low"].shift(1)
data["prv_low_2"] = data["Low"].shift(2)
data["prv_low_3"] = data["Low"].shift(3)
data["prv_low_4"] = data["Low"].shift(4)
data["prv_low_5"] = data["Low"].shift(5)
data["Low_Last_5d"] = data[["prv_low_1", "prv_low_2","prv_low_3","prv_low_4","prv_low_5"]].values.min(1)
data = data.drop(["prv_low_1", "prv_low_2", "prv_low_3", "prv_low_4", "prv_low_5"], axis=1)

# Find the highest price for the stock over the last week
data['prv_high_1'] = data["High"].shift(1)
data['prv_high_2'] = data["High"].shift(2)
data['prv_high_3'] = data["High"].shift(3)
data['prv_high_4'] = data["High"].shift(4)
data['prv_high_5'] = data["High"].shift(5)
data["High_Last_5d"] = data[["prv_high_1", "prv_high_2","prv_high_3","prv_high_4","prv_high_5"]].values.max(1)
data = data.drop(["prv_high_1", "prv_high_2", "prv_high_3", "prv_high_4", "prv_high_5"], axis=1)
data.tail(2)

Price,Close,High,Low,Open,Volume,Low_Last_5d,High_Last_5d
Ticker,1120.SR,1120.SR,1120.SR,1120.SR,1120.SR,,
Date,,,,,,,
2025-06-19,91.199997,91.599998,90.5,91.099998,3742444,88.099998,93.800003
2025-06-22,90.099998,92.199997,90.0,91.199997,2380902,88.099998,93.400002


In [69]:
# Create boolean variable marking whether stock dropped at least six percent in the last week (five days)
data["six_percent_of_close_price"] = .94*data["Close"]
data["Close"] = 1*data["Close"]
data["six_percent_of_close_price_within_last_five_days"] = np.where((data['Low_Last_5d'] <= data['six_percent_of_close_price']) & (data['six_percent_of_close_price'] <= data['High_Last_5d']), True, False)

In [70]:
# Create boolean variable marking whether current close price is greater than yesterdays close price
data["Last_Close"] = data["Close"].shift(1)
data["close_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Last_Close"].values.reshape(data.shape[0])
data["up_from_yesterday"] = np.where((data["close_diff"] > 0), True, False)
data.tail(5)

Price,Close,High,Low,Open,Volume,Low_Last_5d,High_Last_5d,six_percent_of_close_price,six_percent_of_close_price_within_last_five_days,Last_Close,close_diff,up_from_yesterday
Ticker,1120.SR,1120.SR,1120.SR,1120.SR,1120.SR,,,,,,,
Date,,,,,,,,,,,,
2025-06-16,92.599998,93.400002,91.199997,91.199997,3653317,88.099998,95.199997,87.043999,False,91.199997,1.400002,True
2025-06-17,91.699997,93.099998,91.500000,92.599998,3959192,88.099998,95.199997,86.197997,False,92.599998,-0.900002,False
2025-06-18,91.500000,91.699997,90.500000,91.000000,1696730,88.099998,95.199997,86.010000,False,91.699997,-0.199997,False
2025-06-19,91.199997,91.599998,90.500000,91.099998,3742444,88.099998,93.800003,85.727997,False,91.500000,-0.300003,False
2025-06-22,90.099998,92.199997,90.000000,91.199997,2380902,88.099998,93.400002,84.693999,False,91.199997,-1.099998,False


In [71]:
# Buy stock if price dropped at least seven percent in last week and current close price is greater than yesterdays close price
data["buy"] = data["six_percent_of_close_price_within_last_five_days"] & data["up_from_yesterday"]
data["buycumsum"] = data["buy"].cumsum().astype('int64')
data = data.drop(["Last_Close","close_diff","six_percent_of_close_price","up_from_yesterday","six_percent_of_close_price_within_last_five_days","Low_Last_5d","High_Last_5d"], axis=1)

# Add numerical columns of stocks current close price increased and decrease by one percent for final analysis
data["Close_up1%"] = 1.01*data["Close"]
data["Close_down1%"] = .99*data["Close"]
data.tail(10)

Price,Close,High,Low,Open,Volume,buy,buycumsum,Close_up1%,Close_down1%
Ticker,1120.SR,1120.SR,1120.SR,1120.SR,1120.SR,,,,
Date,,,,,,,,,
2025-06-03,90.000000,91.099998,89.800003,91.000000,2996259,False,12,90.900000,89.100000
2025-06-04,93.300003,93.400002,90.099998,90.300003,6143607,False,12,94.233003,92.367003
2025-06-11,93.699997,95.199997,93.199997,94.000000,4913697,True,13,94.636997,92.762997
2025-06-12,92.599998,93.800003,92.000000,93.699997,3442999,False,13,93.525998,91.673998
2025-06-15,91.199997,92.000000,88.099998,88.099998,2784804,False,13,92.111997,90.287997
2025-06-16,92.599998,93.400002,91.199997,91.199997,3653317,False,13,93.525998,91.673998
2025-06-17,91.699997,93.099998,91.500000,92.599998,3959192,False,13,92.616997,90.782997
2025-06-18,91.500000,91.699997,90.500000,91.000000,1696730,False,13,92.415000,90.585000


In [72]:
# view dates where we buy the stock + the two days after
#buyclause = data['buy']==True
#view = data[buyclause.shift(2)|buyclause.shift()|buyclause]
#view

In [73]:
data.loc[(data.buy == False), 'Close_up1%'] = np.nan
data.loc[(data.buy == False), 'Close_down1%'] = np.nan
data.tail(10)

Price,Close,High,Low,Open,Volume,buy,buycumsum,Close_up1%,Close_down1%
Ticker,1120.SR,1120.SR,1120.SR,1120.SR,1120.SR,,,,
Date,,,,,,,,,
2025-06-03,90.000000,91.099998,89.800003,91.000000,2996259,False,12,NaN,NaN
2025-06-04,93.300003,93.400002,90.099998,90.300003,6143607,False,12,NaN,NaN
2025-06-11,93.699997,95.199997,93.199997,94.000000,4913697,True,13,94.636997,92.762997
2025-06-12,92.599998,93.800003,92.000000,93.699997,3442999,False,13,NaN,NaN
2025-06-15,91.199997,92.000000,88.099998,88.099998,2784804,False,13,NaN,NaN
2025-06-16,92.599998,93.400002,91.199997,91.199997,3653317,False,13,NaN,NaN
2025-06-17,91.699997,93.099998,91.500000,92.599998,3959192,False,13,NaN,NaN
2025-06-18,91.500000,91.699997,90.500000,91.000000,1696730,False,13,NaN,NaN


In [74]:
data["Close_up1%"] = data["Close_up1%"].ffill()
data["Close_down1%"] = data["Close_down1%"].ffill()
data.tail(10)

Price,Close,High,Low,Open,Volume,buy,buycumsum,Close_up1%,Close_down1%
Ticker,1120.SR,1120.SR,1120.SR,1120.SR,1120.SR,,,,
Date,,,,,,,,,
2025-06-03,90.000000,91.099998,89.800003,91.000000,2996259,False,12,98.070998,96.128998
2025-06-04,93.300003,93.400002,90.099998,90.300003,6143607,False,12,98.070998,96.128998
2025-06-11,93.699997,95.199997,93.199997,94.000000,4913697,True,13,94.636997,92.762997
2025-06-12,92.599998,93.800003,92.000000,93.699997,3442999,False,13,94.636997,92.762997
2025-06-15,91.199997,92.000000,88.099998,88.099998,2784804,False,13,94.636997,92.762997
2025-06-16,92.599998,93.400002,91.199997,91.199997,3653317,False,13,94.636997,92.762997
2025-06-17,91.699997,93.099998,91.500000,92.599998,3959192,False,13,94.636997,92.762997
2025-06-18,91.500000,91.699997,90.500000,91.000000,1696730,False,13,94.636997,92.762997


In [75]:
data["sellup"] = np.where((data["Close"].values.reshape(data.shape[0])-data["Close_up1%"].values.reshape(data.shape[0]) > 0),True,False)
data["selldown"] = np.where((data["Close_down1%"].values.reshape(data.shape[0])-data["Close"].values.reshape(data.shape[0]) > 0),True,False)
data["sell"] = data["sellup"]|data["selldown"]
data["sellcumsum"] = data["sell"].cumsum().astype('int64')
#data.to_csv('out.csv', index=False)
#data = pd.read_csv('out.csv')
data.tail(10)

Price,Close,High,Low,Open,Volume,buy,buycumsum,Close_up1%,Close_down1%,sellup,selldown,sell,sellcumsum
Ticker,1120.SR,1120.SR,1120.SR,1120.SR,1120.SR,,,,,,,,
Date,,,,,,,,,,,,,
2025-06-03,90.000000,91.099998,89.800003,91.000000,2996259,False,12,98.070998,96.128998,False,True,True,253
2025-06-04,93.300003,93.400002,90.099998,90.300003,6143607,False,12,98.070998,96.128998,False,True,True,254
2025-06-11,93.699997,95.199997,93.199997,94.000000,4913697,True,13,94.636997,92.762997,False,False,False,254
2025-06-12,92.599998,93.800003,92.000000,93.699997,3442999,False,13,94.636997,92.762997,False,True,True,255
2025-06-15,91.199997,92.000000,88.099998,88.099998,2784804,False,13,94.636997,92.762997,False,True,True,256
2025-06-16,92.599998,93.400002,91.199997,91.199997,3653317,False,13,94.636997,92.762997,False,True,True,257
2025-06-17,91.699997,93.099998,91.500000,92.599998,3959192,False,13,94.636997,92.762997,False,True,True,258
2025-06-18,91.500000,91.699997,90.500000,91.000000,1696730,False,13,94.636997,92.762997,False,True,True,259


In [62]:
# Reformat data for final counts
#data.to_csv('out.csv', index=False)
#data = pd.read_csv('out.csv')
#data = data.drop_duplicates(subset="buycumsum",inplace=False)
#print("\n\n"+str(data["closeaboveup1%"].sum()))
#print("\n"+str(data["closebelowdown1%"].sum()))